# List all available datasets


In [1]:
from pycaret.datasets import get_data

datasets = get_data("index", verbose=False)
datasets[datasets["Default Task"] == "Regression"]

,Dataset,Data Types,Default Task,Target Variable 1,Target Variable 2,# Instances,# Attributes,Missing Values
44,automobile,Multivariate,Regression,price,None,202,26,Y
45,bike,Multivariate,Regression,cnt,None,17379,15,N
46,boston,Multivariate,Regression,medv,None,506,14,N
47,concrete,Multivariate,Regression,strength,None,1030,9,N
48,diamond,Multivariate,Regression,Price,None,6000,8,N
49,energy,Multivariate,Regression,Heating Load,Cooling Load,768,10,N
50,forest,Multivariate,Regression,area,None,517,13,N
51,gold,Multivariate,Regression,Gold_T+22,None,2558,121,N
52,house,Multivariate,Regression,SalePrice,None,1461,81,Y
53,insurance,Multivariate,Regression,charges,None,1338,7,N


# Basic workflow


In [2]:
from pycaret.datasets import get_data
from pycaret.regression import RegressionExperiment

data = get_data("forest", verbose=False)

exp = RegressionExperiment()

exp.setup(
    data=data,
    target="area",
    experiment_name="modelling-tools-test-basic-workflow",
    log_experiment=True,
    log_data=True,
    log_plots=True,
    # log_profile=True,
)

,Description,Value
0,Session id,2459
1,Target,area
2,Target type,Regression
3,Original data shape,"(517, 13)"
4,Transformed data shape,"(517, 29)"
5,Transformed train set shape,"(361, 29)"
6,Transformed test set shape,"(156, 29)"
7,Numeric features,10
8,Categorical features,2
9,Preprocess,True


In [3]:
best_methods = exp.compare_models(
    exclude=["lightgbm", "dummy"], sort="RMSE", n_select=3
)

exp.tune_model(best_methods[0], choose_better=True)

exp.ensemble_model(exp.create_model(best_methods[0]), choose_better=True)

exp.stack_models(estimator_list=best_methods, choose_better=True, restack=False)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,13.4934,778.1923,24.7489,-0.2093,1.8657,4.1960,0.0190
omp,Orthogonal Matching Pursuit,13.6025,783.0068,24.9479,-0.2436,1.8646,4.3167,0.0200
lasso,Lasso Regression,13.8319,798.8858,25.2525,-0.2893,1.8847,4.1135,0.0180
llar,Lasso Least Angle Regression,13.8319,798.8875,25.2525,-0.2893,1.8847,4.1135,0.0170
en,Elastic Net,13.8545,798.7551,25.2527,-0.2892,1.8906,4.1176,0.0190
huber,Huber Regressor,9.9178,836.0328,25.2658,-0.1097,1.3895,1.0193,0.0210
ridge,Ridge Regression,14.8362,817.2264,26.0112,-0.6334,1.8883,4.9846,0.0170
par,Passive Aggressive Regressor,11.8659,876.9845,26.3044,-0.2614,1.5245,3.0550,0.0190
lr,Linear Regression,15.2717,826.9306,26.3051,-0.7808,1.9128,5.3912,0.2740
knn,K Neighbors Regressor,15.1395,922.8857,28.0861,-1.0367,1.8915,4.1071,0.0200


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,18.0348,1895.5088,43.5374,-0.0198,1.8595,5.0339
1,10.9912,211.4689,14.5420,-0.0643,1.8426,6.4202
2,9.5488,117.6890,10.8485,-0.2415,1.8731,2.0593
3,16.2251,1225.6877,35.0098,-0.0204,1.8235,1.1869
4,14.0883,586.0330,24.2081,-0.0022,1.9935,4.2258
5,12.0019,393.3985,19.8343,-0.0048,1.8071,4.7301
6,12.8802,704.8072,26.5482,0.0001,1.7838,3.4642
7,9.7099,108.9836,10.4395,-1.6136,2.0880,4.7253
8,9.6495,196.1512,14.0054,-0.0442,1.7532,7.4995


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,17.9903,1891.9614,43.4967,-0.0179,1.8544,5.0653
1,10.8409,210.0883,14.4944,-0.0574,1.8300,6.4449
2,9.6517,121.5485,11.0249,-0.2822,1.8771,2.1113
3,16.4516,1229.4516,35.0635,-0.0235,1.8358,1.1971
4,14.2735,588.0225,24.2492,-0.0056,2.0104,4.1792
5,11.9235,390.8675,19.7704,0.0017,1.7991,4.8155
6,12.8633,703.1515,26.5170,0.0024,1.7777,3.4977
7,9.6983,108.7702,10.4293,-1.6084,2.0873,4.7564
8,9.6554,197.9391,14.0691,-0.0537,1.7496,7.8911


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,18.1105,1892.4452,43.5022,-0.0182,1.8663,5.1319
1,10.3873,209.3732,14.4697,-0.0538,1.7871,6.3431
2,9.9436,129.4326,11.3768,-0.3653,1.8779,2.1559
3,16.1241,1227.5460,35.0364,-0.0219,1.8042,1.1009
4,14.1845,590.4921,24.3000,-0.0098,2.0015,3.8738
5,12.2786,419.4949,20.4816,-0.0714,1.8225,5.1633
6,12.7627,703.9014,26.5311,0.0014,1.7635,3.3722
7,9.0066,101.0271,10.0512,-1.4228,2.0200,3.9470
8,9.6238,203.2973,14.2582,-0.0823,1.7338,8.6671


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,18.2259,1913.5390,43.7440,-0.0295,1.8847,4.8533
1,11.1917,218.9143,14.7958,-0.1018,1.8537,6.5831
2,10.4882,141.9190,11.9130,-0.4971,1.9269,2.4422
3,15.8525,1218.2203,34.9030,-0.0141,1.7950,1.1631
4,13.7348,595.5662,24.4042,-0.0185,1.9498,4.1271
5,12.8282,431.9184,20.7826,-0.1031,1.8707,4.6343
6,12.7969,712.3964,26.6908,-0.0107,1.7746,3.2936
7,9.4331,107.9943,10.3920,-1.5898,2.0606,5.1572
8,9.5331,192.5525,13.8763,-0.0251,1.7430,7.6016


Original model was better than the stacked model, hence it will be returned. NOTE: The display metrics are for the stacked model (not the original one).


BayesianRidge()

In [4]:
exp.automl(optimize="RMSE")

BayesianRidge(alpha_1=0.2, alpha_2=0.05, compute_score=True, lambda_1=0.2,
              lambda_2=1e-07)

# Convenience workflow from `fhdw.modelling`

in the follownig you find a setup with the convenience workflow:


In [5]:
from fhdw.modelling.pycaret import create_regression_model
from pycaret.datasets import get_data

data = get_data("concrete")
exp, best_model = create_regression_model(
    data=data,
    target="strength",
    exclude=["lightgbm", "dummy"],
    save_strategy="local",
    verbose=True,
    prefix="modelling-tools-test-",
    log_experiment=True,
)

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


experiment name: 'modelling-tools-test-strength'


,Description,Value
0,Session id,2309
1,Target,strength
2,Target type,Regression
3,Original data shape,"(1030, 9)"
4,Transformed data shape,"(1030, 9)"
5,Transformed train set shape,"(721, 9)"
6,Transformed test set shape,"(309, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,3.1065,22.5628,4.6978,0.9155,0.1484,0.1096,0.0560
rf,Random Forest Regressor,3.5198,24.3499,4.8856,0.9087,0.1627,0.1284,0.0680
gbr,Gradient Boosting Regressor,3.5989,24.8125,4.9454,0.9067,0.1595,0.1271,0.0310
dt,Decision Tree Regressor,4.6070,52.6686,7.1593,0.7997,0.2312,0.1660,0.0080
ada,AdaBoost Regressor,6.1500,56.3204,7.4838,0.7868,0.2829,0.2673,0.0250
knn,K Neighbors Regressor,6.9951,86.0575,9.1983,0.6758,0.2990,0.2676,0.0110
en,Elastic Net,7.9002,101.7224,10.0477,0.6123,0.3266,0.3125,0.0090
lr,Linear Regression,7.8999,101.7367,10.0483,0.6123,0.3260,0.3118,0.0090
ridge,Ridge Regression,7.8999,101.7366,10.0483,0.6123,0.3260,0.3118,0.0080
lar,Least Angle Regression,7.8999,101.7367,10.0483,0.6123,0.3260,0.3118,0.0070


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.7736,32.2525,5.6791,0.8892,0.1734,0.1269
1,3.4122,32.8305,5.7298,0.8885,0.1504,0.1087
2,2.7290,20.0975,4.4830,0.9082,0.1378,0.1002
3,2.5530,13.2398,3.6387,0.9420,0.1083,0.0797
4,3.4183,23.1914,4.8157,0.9180,0.1591,0.1179
5,3.1336,20.4634,4.5237,0.9307,0.1268,0.0962
6,2.4659,15.7085,3.9634,0.9355,0.1204,0.0861
7,3.3339,25.1584,5.0158,0.8989,0.1363,0.1026
8,2.3980,14.8330,3.8514,0.9421,0.1885,0.1336


...

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4.1464,33.7719,5.8114,0.8840,0.1758,0.1388
1,3.6335,31.9323,5.6509,0.8915,0.1480,0.1137
2,3.1739,20.1087,4.4843,0.9081,0.1444,0.1164
3,2.8305,15.0549,3.8801,0.9341,0.1078,0.0856
4,3.7108,26.1425,5.1130,0.9076,0.1701,0.1333
5,3.4313,23.9870,4.8976,0.9187,0.1392,0.1074
6,3.1770,18.9647,4.3548,0.9221,0.1372,0.1130
7,3.7806,27.1391,5.2095,0.8909,0.1464,0.1227
8,2.8566,18.1284,4.2577,0.9293,0.2090,0.1603


Original model was better than the ensembled model, hence it will be returned. NOTE: The display metrics are for the ensembled model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.6463,31.5713,5.6188,0.8916,0.1808,0.1311
1,3.3781,31.8498,5.6436,0.8918,0.1477,0.1065
2,2.9401,20.4349,4.5205,0.9066,0.1418,0.1106
3,2.6339,11.8180,3.4377,0.9482,0.1143,0.0873
4,3.5045,23.8272,4.8813,0.9158,0.1708,0.1270
5,3.1516,20.8167,4.5625,0.9295,0.1377,0.1035
6,2.6513,15.9558,3.9945,0.9345,0.1294,0.0975
7,3.2905,24.0626,4.9054,0.9033,0.1397,0.1091
8,2.5742,15.1587,3.8934,0.9408,0.2059,0.1532


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.6799,27.3824,5.2328,0.9059,0.1599,0.1188
1,3.3406,28.1184,5.3027,0.9045,0.1359,0.0962
2,2.9493,20.9421,4.5762,0.9043,0.1363,0.0992
3,2.7750,13.2682,3.6426,0.9419,0.1110,0.0858
4,3.4253,21.7241,4.6609,0.9232,0.1477,0.1156
5,3.1403,19.0866,4.3688,0.9353,0.1409,0.1044
6,2.4947,12.9722,3.6017,0.9467,0.1153,0.0850
7,2.9776,20.9445,4.5765,0.9158,0.1297,0.0915
8,2.4027,12.3841,3.5191,0.9517,0.1627,0.1187


Transformation Pipeline and Model Successfully Saved
saved experiment to: 'artifacts/experiments/modelling-tools-test-strength.exp'
saved data to: 'artifacts/data/modelling-tools-test-strength.parquet'
saved best model to: 'artifacts/models/modelling-tools-test-strength.pkl'


# Make Predictions from MLFlow


In [6]:
import mlflow

# following run ID must be changed...
logged_model = "runs:/b1c4b8e1b48d434e8aa4e97b5350ae8d/model"

# TODO: looking for a way to use the "best" model
# this would probably be something about models with 'tags' or 'versions'
# --> do some research here


# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
holdout_data = exp.X_test

loaded_model.predict(holdout_data)

MlflowException: Run 'b1c4b8e1b48d434e8aa4e97b5350ae8d' not found

In [7]:
import mlflow

# from experiment 'modelling-tools-test-strength'
# Experiment ID '904168178551896028'
# Stacking Regressor; Run ID: '5c081b6776d04f7896a63679c60fe9b5'
# Parent Run 'c027affe9cd0482ba35a522115618201'
logged_model = "runs:/5c081b6776d04f7896a63679c60fe9b5/model"


# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
holdout_data = exp.X_test

loaded_model.predict(holdout_data)

2023/12/05 12:20:15 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 1.27.0, required: mlflow==2.8.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


array([27.82910186, 44.03777053, 31.17655482, 11.03272804, 31.22922168,
       41.47641065, 16.21939801, 13.24960513, 39.57687562, 22.97121238,
       43.47331813, 48.40485451, 29.65761608, 24.91474695, 17.7908425 ,
       55.07659447, 53.51716901, 34.99445157, 12.17582496, 40.74179605,
       29.64957374, 20.16480013, 60.20075721, 54.2095764 , 41.22775309,
       33.34146715, 77.93316217, 59.75035805, 78.02371942, 30.73984972,
       77.10326287, 22.87365449, 31.77461089, 71.44127924, 61.46580212,
       35.12758434, 25.12160543, 53.09635336, 22.60585083, 58.19732131,
       24.79360609, 22.98396412, 35.01358694, 52.88923686, 31.35832608,
       25.83091571, 15.09042886, 14.25423594, 24.60267692, 22.45445561,
       13.99028969, 65.19495318, 63.09174455, 26.64628689, 51.20074276,
       23.80006756, 11.25701486, 14.28312907, 20.52490821, 29.03588664,
       14.67341879, 56.77856104, 48.87405053, 33.85032309, 36.64954222,
       32.84451331, 37.11202859, 37.19747393, 25.59461938, 54.28